In [109]:
#@title Imports
import logging
import random
from copy import copy

import random
import platform
from collections import Counter

#from gx_utils import *

In [110]:
#@title Problem instances generator

def problem(N, seed=None):
    """Creates an instance of the problem"""

    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [111]:
from traitlets.traitlets import Callable
import itertools

def remove_duplicates(list_):
  list_ = list(k for k,_ in itertools.groupby(list_))

  return list_

def sort_by(list_: list, key: Callable = None):
  list_.sort(key=key)
  return list_
  
def preproc(problem, rem_dup=False, sort=False, f:Callable = None):
  problem= sort_by(problem, key=f)
  problem= remove_duplicates(problem)
  return problem

In [112]:
#@title Problem init & PreProcessing
N = 5

prob = problem(N, seed=42)
prob = preproc(prob)
prob_dim = len(prob)



In [113]:
check_table = list( True for _ in range(0,21))
sum(check_table)

21

In [116]:
#@title Class & Function Definitions for GA


#You can change this to try different fitness function (FF)
def fitness_function(genome):
    """
    This function compute roughly two metrics: the number of used lists and the percentage of integers covered
    it also uses two parameters (Alpha and Beta) to weigth differently the aforementioned metrics
    """

    #Parameters for weigthing the two metrics used
    alpha = 3
    beta = 2

    #First metric
    num_of_lists_taken = sum(genome)

    check_table = list( False for _ in range(0,N))
    #Read all genes in the genome, using an index
    for index in range(0, len(genome)):

      #if the genome at this index is 1, the individual has the index-th list of the problem
      if genome[index] == 1:

        #so we check for each integer in that list, and we update the checkTable
        for target_integer in range(0,N):
          if target_integer in prob[index]:
            check_table[target_integer] = True
          if all(check_table):
            break

    #Second Metric
    percentage_of_value_covered = ( sum(check_table)/N ) * 100

    #Fitness Score equation  
    fitness_score = alpha*percentage_of_value_covered - beta*num_of_lists_taken  #we want to minimize the second addendum

    return fitness_score


class Individual():

  def __init__(self, genome=None, fitness_score=None):
    self.genome= genome
    self.fitness_score= fitness_score

  def __str__(self):
    return f"""GENOME: {indi.genome}\nFIT_SCORE: {indi.fitness_score}"""

  def evaluate(self):
    """
    This function is mainly used as a wrap function, to let us change only the FF and leave the rest
    of the code untouched, using 'evaluate'. It also helps for readability
    """

    self.fitness_score = fitness_function(self.genome)

  def mutate(self):
    """
    This function simulates mutations over the entire genome. 
    For each gene it tries to mutate with a give probability p.
    The mutation is random between 0 or 1 
    """

    #TODO: check if mutate a single gene with 100% probability is better than this approach

    p = 0.05 #probability of mutation on the single gene

    #for each gene, consider mutate it with a certain probability p
    for gene_index in range(0, len(self.genome)):
      if random.random() < p:
        #set its value randomly
        self.genome[gene_index] = random.choice([0,1])
    self.fitness_score = self.evaluate()

  def cross_over(genome1, genome2):
    #TODO: consider setting a Constant for the length of the genome
    cut_index = random.randint(0,len(genome1))
    temp = []
    temp = genome1[0:cut_index]
    genome1[0:cut_index] = genome2[0:cut_index]
    genome2[0:cut_index] = temp

    return genome1, genome2, cut_index
  
  def recombine_with(self, other_individual: Individual, recomb_function: Callable=cross_over):
    g1 = self.genome
    g2 = other_individual.genome
    
    self.genome, other_individual.genome, cut_index = recomb_function(g1,g2)
    self.evaluate()
    other_individual.evaluate()
    return cut_index

def random_new_individual():
  """
  This function create an Individual object
  Genome defined as a bunch of lists from problem
  Fitness score defined using a FF implemented in the 'evaluate' function
  we take a list of value 1 (list taken) or 0 (list NOT taken), each corresponding to a specific list of the problem
  """
  indi = Individual()
  genome = list(random.choice([0,1]) for _ in range(0, prob_dim))   #something like  [0 1 1 0 ... 0 1 1]
  indi.genome = genome
  indi.evaluate()
  return indi






In [117]:
i1 = random_new_individual()
i2 = random_new_individual()

i1.evaluate
i2.evaluate

#indi.mutate()
#print(f"MUTATED\nGENOME: {indi.genome}")


print(f"BEFORE REC:\n1) {i1}\n\n2) {i2}")
ci=i1.recombine_with(i2)

print(f"AFTER REC at {ci}:\n1) {i1}\n\n2) {i2}")

BEFORE REC:
1) GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0

2) GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
AFTER REC at 1:
1) GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0

2) GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0


In [118]:
for i in range(0,10):
  print(random_new_individual())

#TODO: Understand why the func generates only a single random genome for 10 times and not an entire list of 10 random genomes

GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
GENOME: [1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
FIT_SCORE: 230.0
